In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import librosa
import soundfile as sf

from tqdm.notebook import tqdm
from glob import glob

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchaudio.compliance import kaldi
import torch.nn.functional as F
import json, requests, timm
from collections import defaultdict

pd.set_option('display.max_columns', None)

In [ ]:
class BirdAudioMAEDataset(Dataset):

    def __init__(self, df_audio_meta, dataset_dir, sampling_rate=16_000):
        self.df_audio_meta = df_audio_meta
        self.dataset_dir = dataset_dir
        self.sampling_rate = sampling_rate
        self.global_mean = -4.2677393
        self.global_std = 4.5689974

    def __len__(self):
        return len(self.df_audio_meta)

    def preprocess(self, x: torch.Tensor):
        x = x - x.mean()
        melspec = kaldi.fbank(x.unsqueeze(0), htk_compat=True, window_type="hanning", num_mel_bins=128)
        if melspec.shape[0] < 1024:
            melspec = F.pad(melspec, (0, 0, 0, 1024 - melspec.shape[0]))
        else:
            random_start = np.random.randint(0, len(melspec)-1025)
            melspec = melspec[random_start:random_start+1024]
        melspec = (melspec - self.global_mean) / (self.global_std * 2)
        return melspec

    def __getitem__(self, idx):
        row = self.df_audio_meta.iloc[idx]
        audio_path = f"{self.dataset_dir}/{row['file_name']}"
        # load audio file
        audio_arr, sr = librosa.load(audio_path, sr=self.sampling_rate)
        # get the spectrogram
        spec = self.preprocess(torch.tensor(audio_arr))
        return spec

In [ ]:
TAG = "gaunernst/vit_base_patch16_1024_128.audiomae_as2m_ft_as20k"
MODEL = timm.create_model(f"hf_hub:{TAG}", pretrained=True)

LABEL_URL = "https://huggingface.co/datasets/huggingface/label-files/raw/main/audioset-id2label.json"
AUDIOSET_LABELS = list(json.loads(requests.get(LABEL_URL).content).values())

DEVICE = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

In [ ]:
data_dir = "/workspace/voice_of_jungle/Audio_GreaterManaus"

df_audios = pd.read_csv(f'{data_dir}/metadata.csv')
df_audios = df_audios[df_audios['file_name'].str.contains("wav")]
df_audios['file_exists'] = df_audios['file_name'].apply(lambda x: os.path.exists(f"{data_dir}/{x}"))

# index 8044 is broken
df_audios = df_audios.drop(8044)
df_audios = df_audios.reset_index(drop=True)

df_audios

In [ ]:
batch_size = 1

dataset = BirdAudioMAEDataset(df_audios, data_dir)
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=2)

probs_collect, classes_collect = [], []

MODEL.to(DEVICE)
MODEL.eval()

# ~ 1.5 hours
for i, data in tqdm(enumerate(dataloader), total=len(dataloader)):

    # if i == 0:
    #     print("Example Spectrogram:")
    #     plt.figure(figsize=(9, 2))
    #     plt.imshow(data[0].numpy().T, aspect='auto', origin='lower', cmap='cool')
    #     plt.colorbar()
    #     plt.show()

    data = data.to(DEVICE)
    with torch.no_grad():
        logits = MODEL(data.unsqueeze(1))
        topk_probs, topk_classes = logits.sigmoid().topk(10)

    probs_collect.append(topk_probs.cpu().numpy())
    classes_collect.append(topk_classes.cpu().numpy())

    if (i+1) % 100 == 0:
        print(f"Finished {i+1} batches")

In [ ]:
# aggregate all results
all_probs = np.concatenate(probs_collect)
all_classes = np.concatenate(classes_collect)

class_cols = [f'class_{i+1}' for i in range(10)]
proba_cols = [f'probability_{i+1}' for i in range(10)]

df_results = pd.DataFrame(
    np.concatenate([all_classes, all_probs], axis=1),
    columns = class_cols + proba_cols
)

df_results[class_cols] = df_results[class_cols].astype(int)
df_results[proba_cols] *= 100

id_to_label = {i: v for i, v in enumerate(AUDIOSET_LABELS)}
for _col in class_cols:
    df_results[_col] = df_results[_col].map(id_to_label)
    
df_audios_labels = df_audios.join(df_results, how='left').dropna()

display(df_audios_labels.head())

df_audios_labels.to_csv(f"{data_dir}/Audio_Segment_Labels.csv", index=False)

In [16]:
# df_audios_labels = pd.read_csv(f"{data_dir}/Audio_Segment_Labels.csv")

# df_audios_labels

location          sub_location           location_id  \
0        Inhaa  Inhaa-Be Audiomoth 2               Inhaa_2   
1        Inhaa  Inhaa-Be Audiomoth 2               Inhaa_2   
2        Inhaa  Inhaa-Be Audiomoth 2               Inhaa_2   
3        Inhaa  Inhaa-Be Audiomoth 2               Inhaa_2   
4        Inhaa  Inhaa-Be Audiomoth 2               Inhaa_2   
...        ...                   ...                   ...   
10617    Xingu        inore cot prut  Xingu_Inore_Cot_Prut   
10618    Xingu        inore cot prut  Xingu_Inore_Cot_Prut   
10619    Xingu        inore cot prut  Xingu_Inore_Cot_Prut   
10620    Xingu        inore cot prut  Xingu_Inore_Cot_Prut   
10621    Xingu        inore cot prut  Xingu_Inore_Cot_Prut   

                 timestamp  sampling_rate  \
0      2024-04-05 12:40:00          48000   
1      2024-04-05 12:45:00          48000   
2      2024-04-05 12:50:00          48000   
3      2024-04-05 12:55:00          48000   
4      2024-04-05 13:00:00          48000   
...                    ...            ...   
10617  2023-11-10 22:30:00          48000   
10618  2023-11-10 22:35:00          48000   
10619  2023-11-10 22:40:00          48000   
10620  2023-11-10 22:45:00          48000   
10621  2023-11-10 22:50:00          48000   

                                                mp3_path  \
0      /content/drive/MyDrive/Colab Notebooks/Audio/I...   
1      /content/drive/MyDrive/Colab Notebooks/Audio/I...   
2      /content/drive/MyDrive/Colab Notebooks/Audio/I...   
3      /content/drive/MyDrive/Colab Notebooks/Audio/I...   
4      /content/drive/MyDrive/Colab Notebooks/Audio/I...   
...                                                  ...   
10617  /content/drive/MyDrive/Colab Notebooks/Audio/X...   
10618  /content/drive/MyDrive/Colab Notebooks/Audio/X...   
10619  /content/drive/MyDrive/Colab Notebooks/Audio/X...   
10620  /content/drive/MyDrive/Colab Notebooks/Audio/X...   
10621  /content/drive/MyDrive/Colab Notebooks/Audio/X...   

                                              file_name  file_exists  \
0                    data/Inhaa_2_20240405T124000_5.wav         True   
1                   data/Inhaa_2_20240405T124500_39.wav         True   
2                   data/Inhaa_2_20240405T125000_40.wav         True   
3                    data/Inhaa_2_20240405T125500_3.wav         True   
4                   data/Inhaa_2_20240405T130000_25.wav         True   
...                                                 ...          ...   
10617  data/Xingu_Inore_Cot_Prut_20231110T223000_24.wav         True   
10618  data/Xingu_Inore_Cot_Prut_20231110T223500_42.wav         True   
10619  data/Xingu_Inore_Cot_Prut_20231110T224000_35.wav         True   
10620  data/Xingu_Inore_Cot_Prut_20231110T224500_22.wav         True   
10621  data/Xingu_Inore_Cot_Prut_20231110T225000_15.wav         True   

        class_1          class_2          class_3                    class_4  \
0        Insect          Cricket             Frog                     Animal   
1        Insect          Cricket             Frog        Environmental noise   
2        Insect          Cricket             Frog  Outside, rural or natural   
3       Cricket           Insect           Animal  Outside, rural or natural   
4        Insect          Cricket             Frog  Outside, rural or natural   
...         ...              ...              ...                        ...   
10617   Cricket         Raindrop  Rain on surface                       Rain   
10618    Insect  Rain on surface         Raindrop                       Rain   
10619    Insect          Cricket             Rain                     Animal   
10620  Raindrop             Rain           Insect            Rain on surface   
10621  Raindrop             Rain  Rain on surface                     Insect   

                         class_5                    class_6  \
0                          Croak  Outside, rural or natural   
1      Outside, rural or natural               

In [32]:
fig, ax = plt.subplots(1, 1, figsize=(16, 5))
df_audios_labels['class_1'].value_counts().plot(kind='bar', ax=ax, color='k')
ax.set_ylabel("Count")
ax.grid(axis='y', linestyle='--', alpha=0.6)
ax.set_title("Top-1 Class Distribution")
ax.set_facecolor('#f3ede2')
fig.set_facecolor('#f3ede2')
plt.yscale('log')
plt.show()